# Coding Your Own Linear Regression Model

One task that you will almost certainly be required to do other data science courses (especially if you are a MIDS student) is to write up some of your statistical / machine learning models from scratch. This can be a very valuable exercise, as it ensures that you understand what is actually going on behind the scenes of the models you use ever day, and that you don't just think of them as "black boxes". 

To get a little practice doing this, today you will be coding up your own linear regression model! 

(If you are using this site but aren't actually in this class, you are welcome to skip this exercise if you'd like -- this is more about practicing Python in anticipation of the requirements of other courses than developing your applied data science skills.) 

There are, broadly speaking, two approaches you can take to coding up your own model: 

1. you can write the model by defining a new function, or 
2. you can write the model by defining a new class with associated methods (making a model that works the way a model works in `scikit-learn`). 

Whether you do 1 or 2 is very much a matter of choice and style. Approach one, for example, is more consistent with what is called a *functional* style of programming, while approach two is more consistent with an *object-oriented* style of programming. Python can readily support both approaches, so either would work fine. 

In these exercises, however, I will ask you to use approach number 2 as this *tends* to be the more difficult approach, and so practicing approach 2 will be extra useful in preparing you for other classes (HA! Pun...). In particular, our goal is to implement a linear regression model that has the same "initialize-fit-predict-score" API (application programming interface -- a fancy name for the methods a class supports) as `scikit-learn` models. That means your model should be able to do all of the following:

1. **Initialize** a new model. 
2. **Fit** a linear model when given a numpy vector (`y`) and a numpy matrix (`X`) with the syntax `my_model.fit(X, y)`. 
3. **Predict** values when given a new `numpy` matrix (`X_test`) with the syntax `my_model.predict(X_test)`. 
4. Return the **model coefficients** through the property `my_model.coefficients` (not quite what is used in `sklearn`, but let's use that interface). 

Also, bear in mind that throughout these exercises, we'll be working in `numpy` instead of `pandas`, just as we do in `scikit-learn`. So assume that before using your model, your user has already converted their data from `pandas` into `numpy` arrays. 

**(1)** Define a new Class called `MyLinearModel` with methods for `__init__`, `fit`, `predict`, and an attribute for `coefficients`. For now, we don't need any initialization *arguments*, just an `__init__` function. 

As you get your code outline going, start by just having each method `pass`:

```python
def my_method(self):
    pass
```

This will allow your methods to run without errors (they just don't do anything). Then we can double back to each method to get them working one by one.

In [1]:
# Importing Required Libraries
import numpy as np
import statsmodels.api as sm

In [2]:
class MyLinearModel:
    """Class to Manually Create a Linear Regression Model"""

    def __init__(self):
        self.coefficients = None
        pass

    def fit(self, X, y):
        pass

    def predict(self, X_test):
        pass

**(2)** Now define your `fit` method. This is the method that should actually run your linear regression. In case you've forgotten your linear algebra, remember that for linear regressions, $\beta = (X'X)^{-1}X'Y$, a fact you can see explained in detail on page four [here](https://www.stat.purdue.edu/~boli/stat512/lectures/topic3.pdf).

Note that once you have written the code to do a linear regression, you'll need to put your outputs (your coefficents) somewhere. I recommend making an attribute for your class where you can store your coefficients. 

(As a reminder: the normal multiply operator (`*`) in `numpy` implies scalar multiplication. Use `@` for matrix multiplication). 

**HINT:** Remember that linear regressions require a vector of 1s in the `X` matrix. As the package writer, you get to decide whether users are expected to provide a matrix `X` that already has a vector of 1s, or whether you expect the user to provide a matrix `X` that doesn't have a vector of 1s (in which case you will need to add a vector of 1s before you fit the model).

In [3]:
class MyLinearModel:
    """Class to Manually Create a Linear Regression Model"""

    def __init__(self):
        """Initializing the class and values"""
        self.coefficients = None
        pass

    def fit(self, X, y):
        """To Fit the model"""
        # Adding  1s to input, our class doesn't require user to do this
        inter = np.ones((X.shape[0], 1))
        X = np.hstack((inter, X))

        # Calculating coefficients
        X_T = X.T
        self.coefficients = (np.linalg.inv(X_T @ X)) @ X_T @ y
        pass

    def predict(self, X_test):
        pass

**(3)** As you write code, it is good to test your code as you work. With that in mind, let's create some toy data. First, create a 100 x 2 matrix where each column is normally distributed. Then create a vector `y` that is a linear combination of those two columns **plus** a vector of normally distributed noise **and** a constant term. 

In other words, we want to create data where we *know* exactly what coefficients we should see so when we test our regression, we know if the results are accurate!

In [4]:
# creating sample data for testing
np.random.seed(361)
x1 = np.random.normal(10, 3, 100)
x2 = np.random.normal(50, 20, 100)

X = np.column_stack((x1, x2))

# creating co-efficients and intercept for testing
coef = np.array([2, 3])
intercept = 5

# creating y
noise = np.random.normal(0, 2, 100)
y = X @ coef + intercept + noise

**(4)** Now test whether you `fit` method generates the correct coefficients. Remember the choice you made in Question 2 about whether your package expects the users' `X` matrix to include a vector of 1s when you test!

In [5]:
# Creating a funtions which will initialize our model and fit it
def run_model():
    model = MyLinearModel()
    model.fit(X, y)
    return model

In [6]:
model = run_model()

# Printing the coefficients
print(f"""The Coefficients returnted by the model are: {model.coefficients}""")
print(
    f"""The actual Coefficients used to create the data are: [{intercept},{coef[0]},{coef[1]}]"""
)

The Coefficients returnted by the model are: [6.27879058 1.92874625 2.98361765]
The actual Coefficients used to create the data are: [5,2,3]


## Observations
The Co-efficients retunred by the model are very similar to the actual co-efficients and the difference is due to the noise added to the y variable during data generation.

**(5)** Now let's make the statisticians proud, and in addition to storing the coefficients, let's store the standard errors for our estimated coefficients as another attribute. Recall that the simplest method of calculating the variance covariance matrix for $\beta$ is using the formula $\sigma^2 (X'X)^{-1}$, where $\sigma^2$ is the variance of the error terms of your regression. The standard errors for your coefficient estimates will be the diagonal values of that matrix. See page six [here](https://www.stat.purdue.edu/~boli/stat512/lectures/topic3.pdf) for a full derivation. 

In [7]:
class MyLinearModel:
    """Class to Manually Create a Linear Regression Model"""

    def __init__(self):
        """Initializing the class and values"""
        self.coefficients = None
        self.std_error = None
        pass

    def fit(self, X, y):
        """To Fit the model"""
        # Adding  1s to input, our class doesn't require user to do this
        inter = np.ones((X.shape[0], 1))
        X = np.hstack((inter, X))

        # Calculating coefficients
        X_T = X.T
        self.coefficients = (np.linalg.inv(X_T @ X)) @ X_T @ y

        # calculating standard error
        y_pred = X @ self.coefficients
        error = y - y_pred
        variance = np.var(error)

        variance_mat = variance * np.linalg.inv(X_T @ X)

        self.std_error = np.sqrt(np.diagonal(variance_mat))
        pass

    def predict(self, X_test):
        pass

In [8]:
# testing if the standard error is retunred
model = run_model()
print(f"""The Standard Error returned by the model is: {model.std_error}""")

The Standard Error returned by the model is: [0.75865328 0.05901864 0.00876091]


**(6)** Now let's also add an R-squared attribute to the model.

In [9]:
class MyLinearModel:
    """Class to Manually Create a Linear Regression Model"""

    def __init__(self):
        """Initializing the class and values"""
        self.coefficients = None
        self.std_error = None
        self.r2 = None
        pass

    def fit(self, X, y):
        """To Fit the model"""
        # Adding  1s to input, our class doesn't require user to do this
        inter = np.ones((X.shape[0], 1))
        X = np.hstack((inter, X))

        # Calculating coefficients
        X_T = X.T
        self.coefficients = (np.linalg.inv(X_T @ X)) @ X_T @ y

        # calculating standard error
        y_pred = X @ self.coefficients
        error = y - y_pred
        variance = np.var(error)

        variance_mat = variance * np.linalg.inv(X_T @ X)

        self.std_error = np.sqrt(np.diagonal(variance_mat))

        # calculating R2
        y_mean = np.mean(y)
        tss = np.sum((y - y_mean) ** 2)
        self.r2 = 1 - (np.sum(error**2) / tss)
        pass

    def predict(self, X_test):
        pass

In [10]:
# testing if the R-squared is retunred
model = run_model()
print(f"""The R-squared value returnted by the model is: {model.r2:.3f}""")

The R-squared value returnted by the model is: 0.999


**(7)** Now we'll go ahead and cheat a little. Use `statsmodels` to fit your model with your toy data to ensure your standard errors and r-squared are correct!

In [11]:
# Fitting the model using statsmodels
X_constatnt = sm.add_constant(X)
stats_model = sm.OLS(y, X_constatnt).fit()

In [12]:
# comparing values between our model and statsmodels
print(
    f"""The standard errors returnted by the model are: {model.std_error} and by statsmodels are: {stats_model.bse}"""
)
print(
    f"""The R-squared value returnted by the model is: {model.r2:.3f} and by statsmodels is: {stats_model.rsquared:.3f}"""
)

The standard errors returnted by the model are: [0.75865328 0.05901864 0.00876091] and by statsmodels are: [0.7702957  0.05992435 0.00889535]
The R-squared value returnted by the model is: 0.999 and by statsmodels is: 0.999


## Observations
The Standard errors and R-squared values retured by our model are similar to the values returned by statsmodels.

**(8)** Now implement `predict`! Then test it against your original `X` data -- do you get back something very close to your true `y`?

In [13]:
class MyLinearModel:
    """Class to Manually Create a Linear Regression Model"""

    def __init__(self):
        """Initializing the class and values"""
        self.coefficients = None
        self.std_error = None
        self.r2 = None
        pass

    def fit(self, X, y):
        """To Fit the model"""
        # Adding  1s to input, our class doesn't require user to do this
        inter = np.ones((X.shape[0], 1))
        X = np.hstack((inter, X))

        # Calculating coefficients
        X_T = X.T
        self.coefficients = (np.linalg.inv(X_T @ X)) @ X_T @ y

        # calculating standard error
        y_pred = X @ self.coefficients
        error = y - y_pred
        variance = np.var(error)

        variance_mat = variance * np.linalg.inv(X_T @ X)

        self.std_error = np.sqrt(np.diagonal(variance_mat))

        # calculating R2
        y_mean = np.mean(y)
        tss = np.sum((y - y_mean) ** 2)
        self.r2 = 1 - (np.sum(error**2) / tss)
        pass

    def predict(self, X_test):
        """To predict using the model"""
        inter = np.ones((X_test.shape[0], 1))
        X_test = np.hstack((inter, X_test))
        return X_test @ self.coefficients

In [14]:
y

array([288.18265554,  62.32640271, 165.20303427, 135.98284291,
       259.98148948, 182.15399911, 185.60334636, 224.03192296,
       177.66641095, 106.67192093, 212.23990363, 142.88896975,
       136.24439496, 194.08196445,  94.55385818, 103.76433742,
       142.95347092, 201.36042474, 248.58221802, 219.27315274,
       200.62161035, 302.62523004, 179.81655612, 133.61813511,
       115.90336527, 101.44599085, 203.69087319, 110.06892588,
       231.23688063, 219.34626921,  94.52262172, 135.17867195,
       172.29350997, 176.38496067, 159.34722789, 233.18469331,
        33.67750048, 213.98209911, 200.65509455, 156.53527787,
       281.27354297,  79.33802261, 145.14281964, 150.56307393,
       104.41537345, 134.07224232, 227.76046249, 144.62112424,
       194.31321194,  93.21315277, 124.53658961, 169.92447215,
       204.03683599,  -2.42967444, 164.25382353, 110.46783482,
       236.04906079, 180.56482449, 200.23399689, 131.58314339,
       166.14946685, 269.13021069, 138.92142333, 232.10

In [15]:
model = run_model()
y_model = model.predict(X)
y_model

array([288.63241303,  60.79326344, 165.05591705, 134.32279866,
       262.27149523, 183.75149351, 185.3022875 , 224.8938038 ,
       181.59188267, 107.2822644 , 211.95797281, 145.10789537,
       137.98061944, 195.61407621,  92.73646609, 104.27321151,
       145.09369384, 199.49646425, 246.48589526, 218.60589251,
       199.77054262, 301.66558281, 180.11885578, 132.90850051,
       117.28811076, 101.64493044, 207.56059875, 107.93898956,
       230.26974032, 216.85557254,  93.14341096, 136.33966895,
       171.51036902, 177.29843065, 154.56886607, 231.4711056 ,
        34.17930694, 209.83978321, 201.63589263, 156.89093007,
       279.1105909 ,  77.68776635, 148.94991626, 149.0440879 ,
       104.58839201, 133.74939866, 224.80184914, 146.24791143,
       195.48176728,  94.53073968, 123.75183761, 169.60346391,
       202.34128778,  -2.21067111, 166.4497835 , 110.11270426,
       236.96312957, 177.60812289, 200.98852022, 132.5810755 ,
       167.86171359, 267.00998634, 140.55415591, 229.63

In [16]:
# Comparing Model prediction against original Y values
print(
    f"""Sample y values from our data: {y[:5]} \nThe respective y values from our model: {y_model[:5]}"""
)

Sample y values from our data: [288.18265554  62.32640271 165.20303427 135.98284291 259.98148948] 
The respective y values from our model: [288.63241303  60.79326344 165.05591705 134.32279866 262.27149523]


## Observations
The predicted y values from the model are close to the actual y values which we generated for testing the model.

**(9)** Finally, create the *option* of fitting the model with or without a constant term. In other words, create an option so that, if the user passes a numpy array *without* a constant term, your code will add a vector of 1s before fitting the model. As in `scikit-learn`, make this an option you set during initialization. 

In [17]:
class MyLinearModel:
    """Class to Manually Create a Linear Regression Model"""

    def __init__(self, auto_constant=True):
        """Initializing the class and values"""
        self.coefficients = None
        self.std_error = None
        self.r2 = None
        self.auto_constant = auto_constant
        pass

    def fit(self, X, y):
        """To Fit the model"""
        if self.auto_constant:
            # Adding constant to input if user wants it
            inter = np.ones((X.shape[0], 1))
            X = np.hstack((inter, X))

        # Calculating coefficients
        X_T = X.T
        self.coefficients = (np.linalg.inv(X_T @ X)) @ X_T @ y

        # calculating standard error
        y_pred = X @ self.coefficients
        error = y - y_pred
        variance = np.var(error)

        variance_mat = variance * np.linalg.inv(X_T @ X)

        self.std_error = np.sqrt(np.diagonal(variance_mat))

        # calculating R2
        y_mean = np.mean(y)
        tss = np.sum((y - y_mean) ** 2)
        self.r2 = 1 - (np.sum(error**2) / tss)
        pass

    def predict(self, X_test):
        """To predict using the model"""
        inter = np.ones((X_test.shape[0], 1))
        X_test = np.hstack((inter, X_test))
        return X_test @ self.coefficients

In [18]:
model_auto = MyLinearModel()
model_auto.fit(X, y)
auto_coeff = model_auto.coefficients


model_manual = MyLinearModel(auto_constant=False)
model_manual.fit(X_constatnt, y)
manual_coeff = model_manual.coefficients

print(
    f"""The Coefficients returnted by the model without user including constant term are: {auto_coeff}"""
)
print(
    f"""The Coefficients returnted by the model with user including constant term are: {manual_coeff}"""
)

The Coefficients returnted by the model without user including constant term are: [6.27879058 1.92874625 2.98361765]
The Coefficients returnted by the model with user including constant term are: [6.27879058 1.92874625 2.98361765]


## Observations
The model returns the same coefficients with or without a constant term provided by the user if the user selectes the correct parameters and provides correct data. By default our model adds the constant term.